In [2]:
import Levenshtein as lev
def leven(str1, str2):
    Distance = lev.distance(str1.lower(), str2.lower())
    Ratio = lev.ratio(str1.lower(), str2.lower())
#     print(Ratio)
    if Ratio > .95:
        return True
    else:
        return False

In [3]:
import re
def merge_awards(results):
    A = []
    B = []
    C = []
    for i in range(len(results)):
        if re.search("-",results[i][0]):
            A.append(results[i])
        else:
            B.append(results[i])
    # print(A)
    for a in A:
        str1 = a[0]
        has_merge = False
        for b in B:
            str2 = b[0]
            if leven(str1,str2):
                C.append((str1,a[1]+b[1]))
#                 print(str1,"  ",str2)
                B.remove(b)
                has_merge = True
        if not has_merge:
            C.append(a)
            
                
    for b in B:
        C.append(b)
        
    return C

In [4]:
def filter_awards(results):
    awards = []
    for result in results:
        str1 = result[0]
        has_filter = False
        if awards:
            for award in awards:
                str2 = award[0]
                if leven(str1,str2):
                    has_filter = True
                    break
            if not has_filter:
                awards.append(result)
        else:
            awards.append(result)
    return awards

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

def award_handler(text): 
    filter_words = ['django unchained',]

def str_handler(text):
    size = len(text)
    i = size - 1
    idx = size
    text = text.replace(",","-")
    
    while i >= 0:
        if text[i]<='z' and text[i]>='a':
            idx = i+1 
            break
        i -= 1
    return text[:idx].strip()       

def extract_award(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
#     filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filter_words = ['golden','globes', 'goldenglobe','globes', 'glodenglobes', 'http','2013','2015','2018','2019','2020']
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words and w not in filter_words:
#             w = w.replace('-',",")
            w = w.replace('/',",")
            filtered_sentence.append(w)
    result = ""
    start = False
#     print(filtered_sentence)
#     if re.search('best screenplay - motion picture',text):
#         print(text)
        
    for word in filtered_sentence:    
        if word == 'best':
            start = True
            result += word + " "
            continue
        
        if start and (re.match('w[io]n',word) or re.match('go(es)',word) or re.match('at',word) or re.match('went to',word) or re.match('[;\?:.!@#`]',word)):
            return str_handler(result.strip())

        if start:
            result += word+" "
    return str_handler(result.strip())
#     print(result)

In [30]:
def sanitize(lst):
    level0 = ['screenplay','director','foreign','motion','original','television','tv','animated','song']
    level1 = ['actor','actress']
    level2 = ['supporting','tv','television','motion','series']
    level3 = ['motion','tv','television','drama','comedy','musical']
    results = []
    for r in lst:
        text = r[0]
        if any([kw in text for kw in level1]):
            if any([kw2 in text for kw2 in level2]):
                l = []
                for kw2 in level2:
                    if kw2 in text:
                        l.append(kw2)
                        break
                if any([kw3 in text for kw3 in level3]):
                    for kw3 in level3:
                        if kw3 in text and kw3 not in l:
                            results.append(r)
                            break
                       
        else:
            if any([kw in text for kw in level0]):
                results.append(r)
    return results    
        
# results = [('best actor in motion tv movie', 18),
#  ('best animated feature film', 479),
#  ('best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 461),
#  ('best motion picture - comedy musical', 442)]
# sanitize(results)

In [39]:
import json
import re
import regex
from collections import Counter
tweets = json.load(open("gg2015.json"))
reg = regex.Regex()
search_terms = ['best','w[io]n','go(es)? to','went to',':']
award_names = []
for tweet in tweets:
    text = tweet['text'].lower()
    if 'RT' not in tweet['text']:
        if re.search(search_terms[0],text) and (re.search(search_terms[1],text) or re.search(search_terms[2],text) or re.search(search_terms[3],text) or re.search(search_terms[4],text)):
            extract_str = extract_award(text)
            if re.search('usa',extract_str):
                continue
            if re.search('globe',extract_str):
                continue
            if len(extract_str.split())<4:
                continue
            count = Counter(extract_str)
            if count['-']>1:
                continue
            award_names.append(extract_str)
results = Counter(award_names).most_common()   
results = merge_awards(results)
# results = Counter(results).most_common() 
results = sorted(results, key = lambda x: x[1],reverse = True)
results
results = filter_awards(results)
results = sanitize(results[:70])
print(len(results))
results

44


[('best motion picture - drama', 943),
 ('best animated feature film', 479),
 ('best foreign language film', 461),
 ('best motion picture - comedy musical', 442),
 ('best actor motion picture - drama', 388),
 ('best tv series - drama', 370),
 ('best tv comedy- musical', 308),
 ('best actress motion picture - drama', 301),
 ('best supporting actress - motion picture', 284),
 ('best actor tv series - drama', 280),
 ('best motion picture - musical comedy', 272),
 ('best actress motion picture - comedy musical', 271),
 ('best original song - motion picture', 263),
 ('best tv drama series', 263),
 ('best supporting actor - motion picture', 232),
 ('best actor- tv drama', 199),
 ("best original song 'glory", 198),
 ('best director - motion picture', 194),
 ('best tv comedy series', 157),
 ('best film best director awards', 155),
 ('best actress tv series - drama', 153),
 ('best actress - tv comedy', 147),
 ("best actress - tv drama 'the affair", 136),
 ('best mini-series tv movie', 134),
 ('

In [28]:
file = json.load(open("gg2013answers.json"))
data = file['award_data']
for key in data.keys():
    print(key)

best screenplay - motion picture
best director - motion picture
best performance by an actress in a television series - comedy or musical
best foreign language film
best performance by an actor in a supporting role in a motion picture
best performance by an actress in a supporting role in a series, mini-series or motion picture made for television
best motion picture - comedy or musical
best performance by an actress in a motion picture - comedy or musical
best mini-series or motion picture made for television
best original score - motion picture
best performance by an actress in a television series - drama
best performance by an actress in a motion picture - drama
cecil b. demille award
best performance by an actor in a motion picture - comedy or musical
best motion picture - drama
best performance by an actor in a supporting role in a series, mini-series or motion picture made for television
best performance by an actress in a supporting role in a motion picture
best television serie

In [2]:
OFFICIAL_AWARDS_1819 = ['best motion picture - drama', 'best motion picture - musical or comedy', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best performance by an actress in a motion picture - musical or comedy', 'best performance by an actor in a motion picture - musical or comedy', 'best performance by an actress in a supporting role in any motion picture', 'best performance by an actor in a supporting role in any motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best motion picture - animated', 'best motion picture - foreign language', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best television series - musical or comedy', 'best television limited series or motion picture made for television', 'best performance by an actress in a limited series or a motion picture made for television', 'best performance by an actor in a limited series or a motion picture made for television', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best performance by an actress in a television series - musical or comedy', 'best performance by an actor in a television series - musical or comedy', 'best performance by an actress in a supporting role in a series, limited series or motion picture made for television', 'best performance by an actor in a supporting role in a series, limited series or motion picture made for television', 'cecil b. demille award']

In [3]:
OFFICIAL_AWARDS_1819

['best motion picture - drama',
 'best motion picture - musical or comedy',
 'best performance by an actress in a motion picture - drama',
 'best performance by an actor in a motion picture - drama',
 'best performance by an actress in a motion picture - musical or comedy',
 'best performance by an actor in a motion picture - musical or comedy',
 'best performance by an actress in a supporting role in any motion picture',
 'best performance by an actor in a supporting role in any motion picture',
 'best director - motion picture',
 'best screenplay - motion picture',
 'best motion picture - animated',
 'best motion picture - foreign language',
 'best original score - motion picture',
 'best original song - motion picture',
 'best television series - drama',
 'best television series - musical or comedy',
 'best television limited series or motion picture made for television',
 'best performance by an actress in a limited series or a motion picture made for television',
 'best performanc

In [4]:
len(OFFICIAL_AWARDS_1819)

26